In [101]:
import os
import urllib.request
import requests
import PyPDF2 as ppdf
import pandas as pd
import spacy

In [2]:
from bs4 import BeautifulSoup

In [4]:
docs_page = "https://www.greenbeltmd.gov/government/city-council/minutes-agendas-videos-and-documents"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
           'Accept-Language': 'en-US,en;q=0.5'}
s = requests.sessions.Session()
r = s.get(docs_page, headers=headers)
gmd_content = r.content

In [19]:
soup = BeautifulSoup(gmd_content)

In [29]:
tables = soup.find_all(name="tr")
len(tables)

0

In [45]:
soup

NameError: name 'soup' is not defined

In [5]:
docs_page = "https://greenbeltmd.granicus.com/ViewPublisher.php?view_id=2"
r  = requests.get(docs_page, headers=headers)
r

<Response [200]>

In [6]:
docs_content = r.content
doc_soup = BeautifulSoup(docs_content)
tbl = doc_soup.find_all("table")
len(tbl)

17

In [7]:
tbl[1]

<table class="listingTable" id="archive" summary="This table lists the details of, and links to view the Video and Documents related to Archived Videos as published by City of Greenbelt, MD.  There is one item per table row.">
                        <thead>
                          <tr>
                                                        
                                                                                          <th class="Name listHeader" scope="col">Name</th>
                                                                                                                                                    <th class="Date listHeader" scope="col">Date</th>
                                                                                                                                                    <th class="Duration listHeader" scope="col">Duration</th>
                                                                                                             

In [8]:
rows = tbl[1].find_all("a")
len(rows)

45

In [9]:
this_row = [row for row in rows if "AgendaViewer" in row]

In [10]:
type(rows[0])

bs4.element.Tag

In [11]:
agenda_links = [row["href"] for row in rows if "AgendaViewer" in row["href"]]
len(agenda_links)

15

In [12]:
agenda1_url = f"https://docs.google.com/gview?url=https:{agenda_links[0]}"
agenda1 = s.get(agenda1_url, headers=headers)

In [14]:
agenda1_url

'https://docs.google.com/gview?url=https://greenbeltmd.granicus.com/AgendaViewer.php?view_id=2&clip_id=1799'

In [18]:
realagenda_url = "https://docs.google.com/gview?url=https%3A%2F%2Fgranicus_production_attachments.s3.amazonaws.com%2Fgreenbeltmd%2Fb2d79766c0d56f1de19e1e1c3bad75060.pdf"

In [19]:
realagenda_url in tbl[1]

False

In [23]:
for link in agenda_links:
    print(link)

//greenbeltmd.granicus.com/AgendaViewer.php?view_id=2&clip_id=1799
//greenbeltmd.granicus.com/AgendaViewer.php?view_id=2&clip_id=1798
//greenbeltmd.granicus.com/AgendaViewer.php?view_id=2&clip_id=1796
//greenbeltmd.granicus.com/AgendaViewer.php?view_id=2&clip_id=1795
//greenbeltmd.granicus.com/AgendaViewer.php?view_id=2&clip_id=1794
//greenbeltmd.granicus.com/AgendaViewer.php?view_id=2&clip_id=1793
//greenbeltmd.granicus.com/AgendaViewer.php?view_id=2&clip_id=1792
//greenbeltmd.granicus.com/AgendaViewer.php?view_id=2&clip_id=1791
//greenbeltmd.granicus.com/AgendaViewer.php?view_id=2&clip_id=1790
//greenbeltmd.granicus.com/AgendaViewer.php?view_id=2&clip_id=1789
//greenbeltmd.granicus.com/AgendaViewer.php?view_id=2&clip_id=1788
//greenbeltmd.granicus.com/AgendaViewer.php?view_id=2&clip_id=1787
//greenbeltmd.granicus.com/AgendaViewer.php?view_id=2&clip_id=1786
//greenbeltmd.granicus.com/AgendaViewer.php?view_id=2&clip_id=1785
//greenbeltmd.granicus.com/AgendaViewer.php?view_id=2&clip_id=

In [28]:
agenda1 = s.post(f"https:{link}", headers=headers)
agenda1

<Response [204]>

In [29]:
agenda1.content

b''

In [40]:
x = f"https:{link}"
a = s.post(x)
if a.history:
    for resp in a.history:
        print(resp.status_code, resp.url)

302 https://greenbeltmd.granicus.com/AgendaViewer.php?view_id=2&clip_id=1784


## This is It!

In [41]:
starturl = x
res = urllib.request.urlopen(starturl)
finalurl = res.geturl()
print(finalurl)

https://docs.google.com/gview?url=https%3A%2F%2Fgranicus_production_attachments.s3.amazonaws.com%2Fgreenbeltmd%2F921d005f5049fa622d3de348683ceb0e0.pdf&embedded=true


In [116]:
yr = 2024
for table in tbl[1:]:    
    links = table.find_all("a")
    agenda_links = [link["href"] for link in links if "AgendaViewer" in link["href"]]
    for link in agenda_links:
        try:
            res = urllib.request.urlopen(f"http:{link}")
            pdf_url = res.geturl()
        except:
            res = requests.get(f"http:{link}", headers=headers)
            pdf_url = res.url
        pdf_url = pdf_url.split("&")[0]
        pdf = requests.get(pdf_url, headers=headers)
        content = str(pdf.content)
        try:
            pdf_2_download = content.split("gview?url%3D")[1].split("\\\\")[0]
        except IndexError:
            try:
                print(yr)
                pdf_2_download = requests.utils.unquote(pdf_url.split("gview?url=")[1])
                print(pdf_2_download)
            except IndexError:
                print(pdf_url)
                print(f"Printing content causing IndexError:\n\t{content}")
                continue
        pdf_res = requests.get(pdf_2_download, headers=headers, verify=False)
        filename = f"./docs/GreenbeltAgenda_{yr}_{link[-4:]}.pdf"
        with open(filename, "wb") as pdf_file:
            pdf_file.write(pdf_res.content)
    yr -= 1

2024
https://granicus_production_attachments.s3.amazonaws.com/greenbeltmd/7e3556acb244e0d40b6b8fd3ebbcb6310.pdf


C:\Users\ben97312\Documents\code\notebooks\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'granicus_production_attachments.s3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


2024
https://granicus_production_attachments.s3.amazonaws.com/greenbeltmd/b2d79766c0d56f1de19e1e1c3bad75060.pdf


C:\Users\ben97312\Documents\code\notebooks\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'granicus_production_attachments.s3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


2024
https://granicus_production_attachments.s3.amazonaws.com/greenbeltmd/51d2b1d23f9fd00b3493fb1150fe65720.pdf


...

2009
http://greenbeltmd.granicus.com/GeneratedAgendaViewer.php?view_id=2
Printing content causing IndexError:
	b'\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "https://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head id="Head1"><title>\r\n\t404 - Page Not Found\r\n</title><link href="../App_Themes/404/404.css" type="text/css" rel="stylesheet" /></head>\r\n<script type="text/javascript">\r\nfunction breakOut()\r\n{\r\n    if (top.location != location) {\r\n        top.location.href = document.location.href;\r\n    }\r\n}\r\n</script>\r\n<body onload="breakOut()">\r\n\r\n<h1>Oops! <span>This link does not exist.</span> <a href="javascript:history.go(-1)">&laquo; Go Back</a></h1>\r\n\r\n\r\n\r\n<h3>What happened? <span>List of possible reasons</span></h3>\r\n\r\n<ul>\r\n\t<li>You entered the wrong URL</li>\r\n\t<li>The link to this page is incorrect</li>\r\n\t<li>The link you are trying to visit no longer exist

C:\Users\ben97312\Documents\code\notebooks\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'greenbeltmd.granicus.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\ben97312\Documents\code\notebooks\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'greenbeltmd.granicus.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


2009
http://greenbeltmd.granicus.com/GeneratedAgendaViewer.php?view_id=2
Printing content causing IndexError:
	b'\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "https://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head id="Head1"><title>\r\n\t404 - Page Not Found\r\n</title><link href="../App_Themes/404/404.css" type="text/css" rel="stylesheet" /></head>\r\n<script type="text/javascript">\r\nfunction breakOut()\r\n{\r\n    if (top.location != location) {\r\n        top.location.href = document.location.href;\r\n    }\r\n}\r\n</script>\r\n<body onload="breakOut()">\r\n\r\n<h1>Oops! <span>This link does not exist.</span> <a href="javascript:history.go(-1)">&laquo; Go Back</a></h1>\r\n\r\n\r\n\r\n<h3>What happened? <span>List of possible reasons</span></h3>\r\n\r\n<ul>\r\n\t<li>You entered the wrong URL</li>\r\n\t<li>The link to this page is incorrect</li>\r\n\t<li>The link you are trying to visit no longer exist

2009
http://greenbeltmd.granicus.com/GeneratedAgendaViewer.php?view_id=2
Printing content causing IndexError:
	b'\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "https://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head id="Head1"><title>\r\n\t404 - Page Not Found\r\n</title><link href="../App_Themes/404/404.css" type="text/css" rel="stylesheet" /></head>\r\n<script type="text/javascript">\r\nfunction breakOut()\r\n{\r\n    if (top.location != location) {\r\n        top.location.href = document.location.href;\r\n    }\r\n}\r\n</script>\r\n<body onload="breakOut()">\r\n\r\n<h1>Oops! <span>This link does not exist.</span> <a href="javascript:history.go(-1)">&laquo; Go Back</a></h1>\r\n\r\n\r\n\r\n<h3>What happened? <span>List of possible reasons</span></h3>\r\n\r\n<ul>\r\n\t<li>You entered the wrong URL</li>\r\n\t<li>The link to this page is incorrect</li>\r\n\t<li>The link you are trying to visit no longer exist

2009
http://greenbeltmd.granicus.com/GeneratedAgendaViewer.php?view_id=2
Printing content causing IndexError:
	b'\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "https://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head id="Head1"><title>\r\n\t404 - Page Not Found\r\n</title><link href="../App_Themes/404/404.css" type="text/css" rel="stylesheet" /></head>\r\n<script type="text/javascript">\r\nfunction breakOut()\r\n{\r\n    if (top.location != location) {\r\n        top.location.href = document.location.href;\r\n    }\r\n}\r\n</script>\r\n<body onload="breakOut()">\r\n\r\n<h1>Oops! <span>This link does not exist.</span> <a href="javascript:history.go(-1)">&laquo; Go Back</a></h1>\r\n\r\n\r\n\r\n<h3>What happened? <span>List of possible reasons</span></h3>\r\n\r\n<ul>\r\n\t<li>You entered the wrong URL</li>\r\n\t<li>The link to this page is incorrect</li>\r\n\t<li>The link you are trying to visit no longer exist

### Well, That's Greenbelt City Council's Docs...

This is the first body of documents I intend to get. I'd like to do some basic processing of these files, & before I get there, I need to extract the data from the pdfs (I used PyPDF2). 

Next to retrieve will be the Maryland State Assembly's body of documents. I'll attempt to go back to 2015, which looks like about where these documents were posted using a different method & not using pdf files. We'll see

### Practicing...

In [118]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_md')
doc = nlp("Bill Gates is the CEO of Microsoft")
displacy.render(doc, style='dep')

In [119]:
for token in doc:
    print(token.text, token.lemma_)

Bill Bill
Gates Gates
is be
the the
CEO ceo
of of
Microsoft Microsoft


In [120]:
doc[0].is_sent_start

True

In [122]:
doc.ents

(Bill Gates, Microsoft)

In [126]:
doc_dir = "./docs"
for pdf_file in os.listdir(doc_dir):
    pdf_file = os.path.join(doc_dir, pdf_file)
    s = os.stat(pdf_file)
    if s.st_size == 0:
        os.remove(pdf_file)        

In [128]:
pdf_doc = ppdf.PdfReader(os.path.join(doc_dir, os.listdir(doc_dir)[0]))

In [136]:
pp = pdf_doc.pages

In [137]:
pp.length_function()

1

In [139]:
text = ""
for i in range(len(pdf_doc.pages)):
    p = pdf_doc.pages[i]
    text += p.extract_text()

In [140]:
text

' \nAgenda, Special Charter Meeting, 11/09/2009 \nPage 1 of 1 GREENBELT CITY COUNCIL \nMunicipal Building \nNovember 9, 2009  \nC \nOrganizational Meeting - 38th City Council \nLibrary - 7:00 p.m. \nC  \nCharter Meeting \nCouncil Room - 7:30 p.m. \n \n \n 1. Call to Order - Outgoing Mayor Davis  2. Roll Call  3. Meditation and Pledge of Allegiance to the Flag  4. Certification of Election Count – City Clerk  5. Oath of Office - Council Elect – by Mayor Davis  6. Election of Temporary Chair  7. Election of Mayor  8. Administration of Oath of Office to  Newly Elected Mayor by Peggy Magee, \nClerk of the Circuit Court of Prince George =s County \n 9. Statement by Mayor  10. Election of Mayor Pro Tem  11. Statement by Mayor Pro Tem  12. Statements by Members of Council  13. Other Business  14. Adjournment '

In [ ]:
class GreenbeltDocument:
    def __init__(self, filepath):
        

In [ ]:
docs = []
for os.listdir(doc_dir):
    text = ""
    pdf_file = os.path.join(doc_dir, pdf_file)
    pdf_doc = ppdf.PdfReader(pdf_file)
    for i in range(len(pdf_doc.pages)):
        p = pdf_doc.pages[i]
        text += p.extract_text()
    docs.append(text)